In [1]:
import os
from datetime import datetime
import requests as r
import numpy as np
import geopandas as gp
from osgeo import gdal
import pyproj
from pyproj import Proj
import rasterio as rio
from rasterio.mask import mask
from rasterio.enums import Resampling
from rasterio.shutil import copy
from shapely.ops import transform
import matplotlib.pyplot as plt

In [2]:
# Set Up Working Environment
inDir = os.getcwd()
os.chdir(inDir)

In [3]:
# AUTHENTICATION CONFIGURATION
from netrc import netrc
from subprocess import Popen
from getpass import getpass

urs = 'urs.earthdata.nasa.gov'    # Earthdata URL to call for authentication
prompts = ['Enter NASA Earthdata Login Username \n(or create an account at urs.earthdata.nasa.gov): ',
           'Enter NASA Earthdata Login Password: ']

# Determine if netrc file exists, and if so, if it includes NASA Earthdata Login Credentials
try:
    netrcDir = os.path.expanduser("~/.netrc")
    netrc(netrcDir).authenticators(urs)[0]
    del netrcDir

# Below, create a netrc file and prompt user for NASA Earthdata Login Username and Password
except FileNotFoundError:
    homeDir = os.path.expanduser("~")
    Popen('touch {0}.netrc | chmod og-rw {0}.netrc | echo machine {1} >> {0}.netrc'.format(homeDir + os.sep, urs), shell=True)
    Popen('echo login {} >> {}.netrc'.format(getpass(prompt=prompts[0]), homeDir + os.sep), shell=True)
    Popen('echo password {} >> {}.netrc'.format(getpass(prompt=prompts[1]), homeDir + os.sep), shell=True)
    del homeDir

# Determine OS and edit netrc file if it exists but is not set up for NASA Earthdata Login
except TypeError:
    homeDir = os.path.expanduser("~")
    Popen('echo machine {1} >> {0}.netrc'.format(homeDir + os.sep, urs), shell=True)
    Popen('echo login {} >> {}.netrc'.format(getpass(prompt=prompts[0]), homeDir + os.sep), shell=True)
    Popen('echo password {} >> {}.netrc'.format(getpass(prompt=prompts[1]), homeDir + os.sep), shell=True)
    del homeDir
del urs, prompts

Enter NASA Earthdata Login Username 
(or create an account at urs.earthdata.nasa.gov): ········
Enter NASA Earthdata Login Password: ········


In [4]:
stac = 'https://cmr.earthdata.nasa.gov/stac/' # CMR-STAC API Endpoint
stac_response = r.get(stac).json()            # Call the STAC API endpoint
stac_lp = [s for s in stac_response['links'] if 'LP' in s['title']]  
lp_cloud = r.get([s for s in stac_lp if s['title'] == 'LPCLOUD'][0]['href']).json()
lp_links = lp_cloud['links']

In [5]:
# Remove unnecessary variables
del stac, stac_lp, stac_response
del lp_cloud

In [6]:
lp_search = [l['href'] for l in lp_links if l['rel'] == 'search'][0]  # Define the search endpoint
search_response = r.get(f"{lp_search}").json()                        # Send GET request to retrieve items
lim = 100
search_query = f"{lp_search}?&limit={lim}"    # Add in a limit parameter to retrieve 100 items at a time.
search_response = r.get(search_query).json()  # send GET request to retrieve first 100 items in the STAC collection

print(f"{len(search_response['features'])} items found!")

100 items found!


In [7]:
# field is area inside tile to grab one tile and not overlapping ones
# fieldtile is the area of the tile that sets the extent as the full tile
field = gp.read_file('map.geojson')
fieldShape = field['geometry'][0] # Define the geometry as a shapely polygon
# Bring in the farm field region of interest
fieldtile = gp.read_file('Tile11SLU.geojson')
tileShape = fieldtile['geometry'][0] # Define the geometry as a shapely polygon

In [8]:
bbox = f'{fieldShape.bounds[0]},{fieldShape.bounds[1]},{fieldShape.bounds[2]},{fieldShape.bounds[3]}'  # Defined from ROI bounds
search_query2 = f"{search_query}&bbox={bbox}"  # Add bbox to query
search_response = r.get(search_query2).json()                                                          # Send request
print(f"{len(search_response['features'])} items found!")

56 items found!


In [9]:
date_time = "2020-01-01T00:00:00Z/2021-01-31T23:31:12Z"  # Define start time period / end time period
search_query3 = f"{search_query2}&datetime={date_time}"  # Add to query that already includes bounding_box
search_response = r.get(search_query3).json()            
print(f"{len(search_response['features'])} items found!")

51 items found!


In [10]:
hls_items = search_response['features']  # Start a list of items for our use case

In [11]:
# Search for the HLSS30 items of interest:
#search_query4 = f"{search_query3}&concept_id={s30_id}"
#s30_items = r.get(search_query4).json()['features']

In [12]:
# Search for the HLSL30 items of interest:
#search_query5 = f"{search_query3}&concept_id={l30_id}"
#l30_items = r.get(search_query5).json()['features']

In [13]:
#hls_items = s30_items + l30_items  # Combine the S30 ad L30 items:
#hls_items


In [14]:
#del bbox, date_time, field, lim, lp_links, lp_search, search_query, search_query2, search_query3, search_response, s30_items, l30_items  # Remove search_query4, search_query5

In [15]:
for i, h in enumerate(search_response['features']):
    if h['assets']['browse']['href'].split('/')[5][9:14] == '11SLU':
        print("good", h['assets']['browse']['href'].split('/')[5][9:14] + f" at index {i} is {h['properties']['eo:cloud_cover']}% cloudy.")
    else:
        print("bad", h['assets']['browse']['href'].split('/')[5][9:14])

good 11SLU at index 0 is 2% cloudy.
good 11SLU at index 1 is 19% cloudy.
good 11SLU at index 2 is 1% cloudy.
good 11SLU at index 3 is 14% cloudy.
good 11SLU at index 4 is 24% cloudy.
good 11SLU at index 5 is 86% cloudy.
good 11SLU at index 6 is 0% cloudy.
good 11SLU at index 7 is 3% cloudy.
good 11SLU at index 8 is 0% cloudy.
good 11SLU at index 9 is 3% cloudy.
good 11SLU at index 10 is 58% cloudy.
good 11SLU at index 11 is 64% cloudy.
good 11SLU at index 12 is 0% cloudy.
good 11SLU at index 13 is 0% cloudy.
good 11SLU at index 14 is 0% cloudy.
good 11SLU at index 15 is 1% cloudy.
good 11SLU at index 16 is 86% cloudy.
good 11SLU at index 17 is 4% cloudy.
good 11SLU at index 18 is 0% cloudy.
good 11SLU at index 19 is 0% cloudy.
good 11SLU at index 20 is 14% cloudy.
good 11SLU at index 21 is 21% cloudy.
good 11SLU at index 22 is 3% cloudy.
good 11SLU at index 23 is 1% cloudy.
good 11SLU at index 24 is 0% cloudy.
good 11SLU at index 25 is 2% cloudy.
good 11SLU at index 26 is 51% cloudy.
g

In [16]:
# GDAL configurations used to successfully access LP DAAC Cloud Assets via vsicurl 
gdal.SetConfigOption("GDAL_HTTP_UNSAFESSL", "YES")
gdal.SetConfigOption('GDAL_HTTP_COOKIEFILE','~/cookies.txt')
gdal.SetConfigOption('GDAL_HTTP_COOKIEJAR', '~/cookies.txt')
gdal.SetConfigOption('GDAL_DISABLE_READDIR_ON_OPEN','YES')
gdal.SetConfigOption('CPL_VSIL_CURL_ALLOWED_EXTENSIONS','TIF')

In [17]:
h = hls_items[0]
#h

In [18]:
evi_band_links = []

# Define which HLS product is being accessed
if h['assets']['browse']['href'].split('/')[4] == 'HLSS30.015':
    evi_bands = ['B8A', 'Fmask'] # NIR for S30
else:
    evi_bands = ['B05', 'Fmask'] # NIR for L30

# Subset the assets in the item down to only the desired bands
for a in h['assets']: 
    if any(b == a for b in evi_bands):
        evi_band_links.append(h['assets'][a]['href'])
for e in evi_band_links: print(e)

https://lpdaac.earthdata.nasa.gov/lp-prod-protected/HLSS30.015/HLS.S30.T11SLU.2020192T183921.v1.5.Fmask.tif
https://lpdaac.earthdata.nasa.gov/lp-prod-protected/HLSS30.015/HLS.S30.T11SLU.2020192T183921.v1.5.B8A.tif


In [19]:
# Use vsicurl to load the data directly into memory (be patient, may take a few seconds)
for e in evi_band_links:
    if e.rsplit('.', 2)[-2] == evi_bands[0]: # NIR index
        nir = rio.open(e)
    elif e.rsplit('.', 2)[-2] == evi_bands[1]: # Fmask index
        fmask = rio.open(e)
    else:
        print('none')
print("The COGs have been loaded into memory!")

The COGs have been loaded into memory!


In [20]:
geo_CRS = Proj('+proj=longlat +datum=WGS84 +no_defs', preserve_units=True)  # Source coordinate system of the ROI
utm = pyproj.Proj(nir.crs)                                                  # Destination coordinate system
project = pyproj.Transformer.from_proj(geo_CRS, utm)                        # Set up the transformation
fsUTM = transform(project.transform, tileShape)                            # Apply reprojection

In [21]:
fmask_array, _ = rio.mask.mask(fmask, [fsUTM], crop=False)  # Load in the Quality data
bitword_order = (1, 1, 1, 1, 1, 1, 2)  # set the number of bits per bitword
num_bitwords = len(bitword_order)      # Define the number of bitwords based on your input above
total_bits = sum(bitword_order)        # Should be 8, 16, or 32 depending on datatype

In [22]:
qVals = list(np.unique(fmask_array))  # Create a list of unique values that need to be converted to binary and decoded
all_bits = list()
goodQuality = []
for v in qVals:
    all_bits = []
    bits = total_bits
    i = 0
    
    # Convert to binary based on the values and # of bits defined above:
    bit_val = format(v, 'b').zfill(bits)
    #print('\n' + str(v) + ' = ' + str(bit_val))
    all_bits.append(str(v) + ' = ' + str(bit_val))

    # Go through & split out the values for each bit word based on input above:
    for b in bitword_order:
        prev_bit = bits
        bits = bits - b
        i = i + 1
        if i == 1:
            bitword = bit_val[bits:]
            #print(' Bit Word ' + str(i) + ': ' + str(bitword))
            all_bits.append(' Bit Word ' + str(i) + ': ' + str(bitword)) 
        elif i == num_bitwords:
            bitword = bit_val[:prev_bit]
            #print(' Bit Word ' + str(i) + ': ' + str(bitword))
            all_bits.append(' Bit Word ' + str(i) + ': ' + str(bitword))
        else:
            bitword = bit_val[bits:prev_bit]
            #print(' Bit Word ' + str(i) + ': ' + str(bitword))
            all_bits.append(' Bit Word ' + str(i) + ': ' + str(bitword))
            
    # 2, 4, 5, 6 are the bits used. All 4 should = 0 if no clouds, cloud shadows were present, and pixel is not snow/ice/water
    if int(all_bits[2].split(': ')[-1]) + int(all_bits[4].split(': ')[-1]) + \
    int(all_bits[5].split(': ')[-1]) + int(all_bits[6].split(': ')[-1]) == 0:
        goodQuality.append(v)

In [23]:
def ndvi(red, nir, Denom):
      return ((nir - red) / (Denom))

In [ ]:
import datetime
t1 = datetime.datetime.now()
print (t1.strftime("%Y-%m-%d %H:%M:%S"))
# Now put it all together and loop through each of the files, visualize, calculate statistics on EVI, and export
for j, h in enumerate(hls_items):
    try:
        timetest = datetime.datetime.now()
        print (timetest.strftime("%Y-%m-%d %H:%M:%S"))
        evi_band_links = []
        if h['assets']['browse']['href'].split('/')[4] == 'HLSS30.015':
            evi_bands = ['B8A', 'B04', 'B02', 'Fmask'] # NIR RED BLUE FMASK
        else:
            evi_bands = ['B05', 'B04', 'B02', 'Fmask'] # NIR RED BLUE FMASK
        for a in h['assets']: 
            if any(b == a for b in evi_bands):
                evi_band_links.append(h['assets'][a]['href'])
        print('band')
        # Use vsicurl to load the data directly into memory (be patient, may take a few seconds)
        for e in evi_band_links:
            if e.rsplit('.', 2)[-2] == evi_bands[0]: # NIR index
                nir = rio.open(e)
            elif e.rsplit('.', 2)[-2] == evi_bands[1]: # red index
                red = rio.open(e)
            #elif e.rsplit('.', 2)[-2] == evi_bands[2]: # blue index
                #blue = rio.open(e)
            elif e.rsplit('.', 2)[-2] == evi_bands[3]: # fmask index
                fmask = rio.open(e)
        print('cog')
        # load data and scale
        nir_array,nir_transform = rio.mask.mask(nir,[fsUTM], crop=False)
        red_array, red_transform = rio.mask.mask(red,[fsUTM], crop=False)
        #blue_array, blue_transform = rio.mask.mask(blue,[fsUTM], crop=False)
        nir_scaled = nir_array[0] * nir.scales[0]
        red_scaled = red_array[0] * red.scales[0]
        #blue_scaled = blue_array[0] * blue.scales[0]
        #nir_scaled[nir_array[0]==nir.nodata] = np.nan 
        #red_scaled[red_array[0]==red.nodata] = np.nan 
        #blue_scaled[blue_array[0]==blue.nodata] = np.nan 
        print('null')
        # Generate EVI
        Denom = red_scaled + nir_scaled
        evi_scaled = ndvi(red_scaled, nir_scaled, Denom)
        print('ndvi')
        # Quality Filter the data
        fmask_array, _ = rio.mask.mask(fmask,[fsUTM], crop=False)
        print('fmask')
        qVals = list(np.unique(fmask_array))
        all_bits = list()
        goodQuality = []
        for v in qVals:
            all_bits = []
            bits = total_bits
            i = 0
            # Convert to binary based on the values and # of bits defined above:
            bit_val = format(v, 'b').zfill(bits)
            all_bits.append(str(v) + ' = ' + str(bit_val))

            # Go through & split out the values for each bit word based on input above:
            for b in bitword_order:
                prev_bit = bits
                bits = bits - b
                i = i + 1
                if i == 1:
                    bitword = bit_val[bits:]
                    all_bits.append(' Bit Word ' + str(i) + ': ' + str(bitword)) 
                elif i == num_bitwords:
                    bitword = bit_val[:prev_bit]
                    all_bits.append(' Bit Word ' + str(i) + ': ' + str(bitword))
                else:
                    bitword = bit_val[bits:prev_bit]
                    all_bits.append(' Bit Word ' + str(i) + ': ' + str(bitword))

            # 2, 4, 5, 6 are the bits used. All should = 0 if no clouds, cloud shadows were present & pixel is not snow/ice/water
            if int(all_bits[2].split(': ')[-1]) + int(all_bits[4].split(': ')[-1]) + \
            int(all_bits[5].split(': ')[-1]) + int(all_bits[6].split(': ')[-1]) == 0:
                goodQuality.append(v)
        evi_band = np.ma.MaskedArray(evi_scaled, np.in1d(fmask_array, goodQuality, invert=True))  # Apply QA mask to the EVI data
        evi_band = np.ma.filled(evi_band, np.nan)
        
        # Remove any observations that are entirely fill value
        if np.nansum(evi_band) == 0.0:
            print(f"File: {h['assets']['browse']['href'].split('/')[-1].rsplit('.', 1)[0]} ({h['id']}) was entirely fill values and will not be exported.")
            continue
        outName = f"{nir.name.rsplit('/', 1)[-1].split('.B')[0]}_EVI.tif"
        tempName = "temp.tif"
        # Create output GeoTIFF with overviews
        evi_tif = rio.open(tempName, 'w', driver='GTiff', height=evi_band.shape[0], width=evi_band.shape[1], count=1,
                           dtype=str(evi_band.dtype), crs=nir.crs, transform=nir_transform)
        evi_tif.write(evi_band, 1)
        evi_tif.build_overviews([2, 4, 8], Resampling.average)
        evi_tif.update_tags(ns='rio_overview', resampling='average')
        
        # Copy the profile, add tiling and compression
        kwds = evi_tif.profile
        kwds['tiled'] = True
        kwds['compress'] = 'LZW'
        evi_tif.close()
        
        # Open temp file and export as valid cog
        with rio.open(tempName, 'r+') as src:
            copy(src, outName, copy_src_overviews=True, **kwds)
        src.close(), os.remove(tempName);

    except:
        print(f"Unable to access file: {h['assets']['browse']['href'].split('/')[-1].rsplit('.', 1)[0]} ({h['id']})")
    print(f"Processing file {j+1} of {len(hls_items)}")
    
t2 = datetime.datetime.now()
time = t2 -t1
print(time)

2021-02-12 21:45:19
2021-02-12 21:45:19
band
cog
null
ndvi
fmask
Processing file 1 of 51
2021-02-12 21:45:38
band
cog
null
ndvi


<ipython-input-23-21176e722086>:2: RuntimeWarning: divide by zero encountered in true_divide
  return ((nir - red) / (Denom))
<ipython-input-23-21176e722086>:2: RuntimeWarning: invalid value encountered in true_divide
  return ((nir - red) / (Denom))


fmask
Processing file 2 of 51
2021-02-12 21:46:08
band
cog
null
ndvi
fmask
Processing file 3 of 51
2021-02-12 21:54:17
band
cog
null
ndvi
fmask
Processing file 4 of 51
2021-02-12 21:54:45
band
cog
null
ndvi
fmask
Processing file 5 of 51
2021-02-12 21:55:12
band
cog
null
ndvi
fmask
Processing file 6 of 51
2021-02-12 22:05:16
band
cog
null
ndvi
fmask
Processing file 7 of 51
2021-02-12 22:05:44
band
cog
null
ndvi
fmask
Processing file 8 of 51
2021-02-12 22:16:42
band
cog
null
ndvi
fmask
Processing file 9 of 51
2021-02-12 22:17:09
band
cog
null
ndvi
fmask
Processing file 10 of 51
2021-02-12 22:27:38
band
cog
null
ndvi
fmask
Processing file 11 of 51
2021-02-12 22:28:04
band
cog
null
ndvi
fmask
Processing file 12 of 51
2021-02-12 22:38:41
band
cog
null
ndvi
fmask
Processing file 13 of 51
2021-02-12 22:39:08
band
cog
null
ndvi
fmask
Processing file 14 of 51
2021-02-12 22:49:53
band
cog
null
ndvi
fmask
Processing file 15 of 51
2021-02-12 22:50:23
band
cog
null
ndvi
fmask
Processing file 16 of 

In [ ]:
# Remove variables that are no longer needed and close the files that were read in memory
del all_bits, b, bit_val, bits, bitword, bitword_order, blue_array, blue_scaled, e, evi_band, evi_band_links, evi_bands
del evi_scaled, fmask_array, goodQuality, h, hls_items, i, nir_array, nir_scaled, nir_transform, num_bitwords, outName
del prev_bit, qVals, red_array, red_scaled, stac, total_bits, v
nir.close(), red.close(), fmask.close()